In [ ]:
# en cours

# Calibration du modèle

Calibration des scores du classifieur

Le classifieur linéaire utilisé (LinearSVC) produit des scores de décision correspondant à la distance à l’hyperplan de séparation, mais ces scores ne sont pas interprétables comme des probabilités. En particulier, leur échelle et leur distribution dépendent fortement de la régularisation et des données d’entraînement.

Dans le cadre de stratégies d’ensemble telles que le stacking ou le blending, il est essentiel que les sorties des différents modèles soient comparables et correctement calibrées. Une mauvaise calibration peut conduire le méta-modèle à sur- ou sous-pondérer certains modèles, indépendamment de leur performance réelle.

Pour cette raison, une calibration a été appliquée au classifieur SVM à l’aide de CalibratedClassifierCV, permettant de transformer les scores de décision en probabilités calibrées via une régression logistique (méthode sigmoid). Cette étape n’a pas pour objectif principal d’améliorer le score de classification, mais d’obtenir des probabilités plus fiables et comparables entre modèles.

In [1]:
svm = # charger svm

calibrated_svm = CalibratedClassifierCV(
    svm,
    method="sigmoid",
    cv=3
)

SyntaxError: invalid syntax (1049053284.py, line 1)